# Setting up

Google Colab lets us access a remote machine to run the protocol rather than our local computer in the form of a **Jupyter Notebook**, a web-based interactive development environment that combines markdown text with Python and Shell commands to perform the analysis. Code can be split into 'chunks' that can be run independently, yet all user-created variables are stored in the notebook's memory to use across different cells. Each time we start a runtime session we need to install and configure the software to use.

## Installing MDAnalysis

**MDAnalysis** is a powerful python library to read and process particle-based trajectories in many different formats, including PDB. It provides a fast and user-friendly framework to manipulate and analyze molecular dynamics simulations.

Install MDAnalysis in you working python environment.

In [ ]:
!pip install MDAnalysis

## Analysis

For this project we will carry out a contacts analysis between the two monomers. By 'contacts' we refer to 'atomic' interactions ocurring during the course of the MD simulation. Protein activity and mechanism of action at the molecular level are fundamentally governed by networks of atom-atom interactions, so it is extremely valuable to know which of these are the most significant and to understand how they change for different states of the system (e.g. activation).

First, load the topology, coordinates (extracted from a PDB file) and trajectory (XTC) of our system to a `MDAnalysis.Universe()`, the most fundamental object in MDAnalysis.

In [ ]:
import MDAnalysis as mda

u = mda.Universe('step5_charmm2gmx.pdb', 'step7_production.xtc')

Now the variable `u` holds the mda.Universe() we just created. Notice that we can access its contents from a different code cell.

In [ ]:
print(u.atoms)

### Number of contacts


First, we define a `get_interactions()` python function, a block of code that we can call anytime to be executed within the notebook. Here, we want the `get_interactions()` function return a list of all the 'atom' interaction pairs between protomers A and B (the two apelin receptors) that occur every trajectory frame. The criterion used to discriminate between interacting and non-interacting atoms will be the euclidian distance. Since it is coarse-grained, we will only retrieve the `resids` to simplify things, which correspond to the residue number within the sequence (atom agnostic).

In [ ]:
import numpy as np
import pandas as pd

def get_interactions(protA, protB):

  interactions = pd.DataFrame()
  for i in protA.atoms:
    for j in protB.atoms:
      distance = np.linalg.norm(i.position - j.position)
      if distance < 5:
        interactions = pd.concat([interactions, pd.DataFrame({'residA': [int(i.resid)], 'residB': [int(j.resid)], 'frame': [ts.frame]})], ignore_index=True)

  return interactions.drop_duplicates()

> Note: This function iterates over all 'atoms' of `protA` and `protB` and computes the euclidean distance between each pair. If the distance is below some threshold then store the residue numbers of each atom involved and trajectory frame where it occurs in a `pandas.DataFrame()` object. Finally, return the DataFrame() object without any duplicate rows (multiple atoms can contribute to the same residue-residue contact).

Now let's calculate how many contacts occur for each trajectory frame. This is a quick way to assess whether the interaction between the two protomers is stable or labile in time.

In [ ]:
ContactsPerFrame = pd.DataFrame()

for ts in u.trajectory[::10]:
    protA = u.select_atoms('segid PROD')
    protB = u.select_atoms('segid PROE and around 10 segid PROD')

    interactions = get_interactions(protA, protB)
    print(f"Frame: {ts.frame}")

    ContactsPerFrame = pd.concat([ContactsPerFrame, pd.DataFrame({'frame': [ts.frame], 'contacts': [len(interactions)]})], ignore_index=True)

> Note: Here we iterate over every 10th frame in the loaded trajectory and call the `get_interactions()` function. Notice that we define `protB` as a subset of the other protomer (those within 10 A distance of `protA`) to save in computational time. Then store the total number of interactions that occur every frame in `ContactsPerFrame`.

In [ ]:
print(ContactsPerFrame)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(ContactsPerFrame['frame'], ContactsPerFrame['contacts'])
plt.xlabel("Frame")
plt.ylabel("Number of contacts")
plt.title("Time evolution of contacts")

plt.show()

> Quesition: Do you think the interaction between the two protomers is stable? Hint: Large swings in total number of contacts is telling of poor stability.

## Contact frequency

Next, let's find out which specific amino acids are involved the most in the contact between the two protomers. For this, we need to get how many times a specific residue-residue interaction occurs throughout the simulation.

In [ ]:
ContactHistory = pd.DataFrame()

for ts in u.trajectory[::10]:
    protA = u.select_atoms('segid PROD')
    protB = u.select_atoms('segid PROE and around 10 segid PROD')

    interactions = get_interactions(protA, protB)
    print(f"Frame: {ts.frame}")

    ContactHistory = pd.concat([ContactHistory, interactions], ignore_index=True)

> Note: Again, we iterate over every 10th frame of the trajectory and call the `get_interactions()` function. This time, we will keep track of all the ocurring interactions for every frame, stored in `ContactHistory`.

In [ ]:
print(ContactHistory)

Now, let's count how many times each unique `residA-residB` interaction occurs. To better display the results, we will construct a *heatmap*, where hotspots of interaction are colored vibrantly in the same spatial region.

In [ ]:
uniqueA = np.sort(ContactHistory['residA'].unique())
uniqueB = np.sort(ContactHistory['residB'].unique())

ContactHeatmap = pd.DataFrame(0, index=uniqueA, columns=uniqueB)

for index, row in ContactHistory.iterrows():
  residA = row['residA']
  residB = row['residB']
  ContactHeatmap.loc[residA, residB] = ContactHeatmap.loc[residA, residB] + 1

> Note: First we generate an empty NxM array as `ContactHeatmap`, where N and M are any residues of protA or protB respectively that have interacted at least once during the simulation. Then, for every interaction that has occured in `ContactHistory` we add 1 count to the `ContactHeatmap` at the `(residA, residB)` location.

In [ ]:
import seaborn as sns

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,8), sharex=True, sharey=True)

sns.heatmap(ContactHeatmap, cmap='Blues', annot=True, cbar=False)
plt.title('Contact heatmap')
plt.ylabel('protA residue number')
plt.xlabel('protB residue number')

plt.show()

> Question: Which residue-residue interactions are the most prevalent?